In [38]:
import os
from werkzeug.utils import secure_filename
from deepface import DeepFace
from pathlib import Path
from deepface.basemodels import VGGFace
from pathlib import Path
import json
import requests

In [36]:
# Rest api end points
ROOT_SENTI_API_URI = "http://20.102.100.20:5000/facial-senti-api"
RAW_SENTI_API_URI = ROOT_SENTI_API_URI + "/raw_sentiments"
headers_ = {'Content-Type': 'application/json'}

In [30]:
model = VGGFace.loadModel()

In [49]:
img_path = "C:\\Users\\anurag\\Documents\\MyProjects\\PythonProjects\\facenet_face-recognition\\photos\\pranali\\1641816597.113409_face.jpg"

df = DeepFace.find(img_path = img_path, db_path = "./dataset", model_name = 'VGG-Face', model = model, distance_metric = 'cosine', enforce_detection=False)
# print(df.head())
if (df.shape[0] > 0):
    print("df.iloc[0]['VGG-Face_cosine'] -> ", df.iloc[0]['VGG-Face_cosine'])
    if (df.iloc[0]['VGG-Face_cosine'] < 0.18):
        obj = DeepFace.analyze(img_path =img_path, actions=['age', 'gender', 'race', 'emotion'], enforce_detection=False)
        emotion_dict = obj['emotion']
        print("emotion_dict: ", emotion_dict)
        
        identity = df.iloc[0].identity.split("\\")[1].split("/")[0]
        print("identity: " , identity)
        json_predsDict = obj['emotion']
        emp_name_n_id = identity.split("-",1) # setting the maxsplit parameter to 1, will return a list with 2 elements!

        sorted_emotion_dict = dict(sorted(emotion_dict.items(), key = lambda x: x[1], reverse=True)) 
        print("sorted_emotion_dict: ", sorted_emotion_dict)
        
        list_sorted_emotion_dict = list(sorted_emotion_dict)
        
        # Note: order of elif block is important here
        if list_sorted_emotion_dict[0] == "happy":
            label = "LIKELY_HAPPY"
        elif (list_sorted_emotion_dict[0] == "neutral") and (list_sorted_emotion_dict[1] == "happy"):
            label = "LIKELY_HAPPY"
        elif list_sorted_emotion_dict[0] == "surprise" and list_sorted_emotion_dict[1] == "happy":
            label = "LIKELY_HAPPY" 
        elif list_sorted_emotion_dict[0] == "neutral" and list_sorted_emotion_dict[5] == "happy":
            label = "LIKELY_NEUTRAL"
        else:
            label = "LIKELY_NOT_HAPPY"
            
        print('label: ', label)
        
        try:
            # predsDict_ = json.dumps(json_predsDict, indent=4)
            data_ = {
                "emotion_scores": json_predsDict,
                "date": '2021-12-19',
                "time": '00:46:44',
                "emp_name": emp_name_n_id[0],
                "emp_id": emp_name_n_id[1],
                "overall_sentiment": label,
                "device_id": 'tsystem-PU-5-r1-device1'
            }

            data_ = json.dumps(data_, indent=4)

            print ("data: ", data_)
            r = requests.post(url = RAW_SENTI_API_URI, data = data_, headers = headers_)
            print ("Return of post request", r.json())
        except Exception as e:
            print ("Problem while making post request to url ", RAW_SENTI_API_URI, ", Problem: ", e)



Already built model is passed
There are  82  representations found in  representations_vgg_face.pkl
find function lasts  0.5130131244659424  seconds
df.iloc[0]['VGG-Face_cosine'] ->  0.14904828924157953


Action: emotion: 100%|██████████| 4/4 [00:01<00:00,  3.97it/s]

emotion_dict:  {'angry': 0.041386380326002836, 'disgust': 1.6212865716624947e-05, 'fear': 87.87886500358582, 'happy': 0.05065537989139557, 'sad': 9.41755473613739, 'surprise': 0.0005752152446802938, 'neutral': 2.6109427213668823}
identity:  Pranali_Khilari-2020002319
sorted_emotion_dict:  {'fear': 87.87886500358582, 'sad': 9.41755473613739, 'neutral': 2.6109427213668823, 'happy': 0.05065537989139557, 'angry': 0.041386380326002836, 'surprise': 0.0005752152446802938, 'disgust': 1.6212865716624947e-05}
label:  LIKELY_NOT_HAPPY
data:  {
    "emotion_scores": {
        "angry": 0.041386380326002836,
        "disgust": 1.6212865716624947e-05,
        "fear": 87.87886500358582,
        "happy": 0.05065537989139557,
        "sad": 9.41755473613739,
        "surprise": 0.0005752152446802938,
        "neutral": 2.6109427213668823
    },
    "date": "2021-12-19",
    "time": "00:46:44",
    "emp_name": "Pranali_Khilari",
    "emp_id": "2020002319",
    "overall_sentiment": "LIKELY_NOT_HAPPY",
   